In this notebook, we impute missing / outlier data and seperate normal and adversarial trips

In [1]:
import pandas as pd
import numpy as np
import datetime
from pykalman import KalmanFilter

In [2]:
df = pd.read_csv("../../df_cleaned_location.csv")

In [3]:
df.drop(["CARGO", "CARGO_PAX", "PAX", 'TRACK_MADE_GOOD', 
        'ENGINE_1_FLOWRATE', 'ENGINE_1_FLOWRATEA', 'ENGINE_1_FLOWRATEB', 'ENGINE_1_FLOWTEMPB',
        'ENGINE_2_FLOWRATE', 'ENGINE_2_FLOWRATEA', 'ENGINE_2_FLOWRATEB','ENGINE_2_FLOWTEMPB'
        ], inplace=True, axis=1)

In [4]:
df["WIND_ANGLE"] = df["WIND_ANGLE"].apply(lambda x: x-360 if x>360 else x) 
df = df[df.trip_id!=0]

In [5]:
cols = list(df.columns)
remove_list = ["Dati", "Time", "HEADING", "LONGITUDE", "LATITUDE", "WIND_ANGLE", "WIND_ANGLE_TRUE", "WIND_SPEED",
               "trip_id", "DEPTH", "PITCH_1", "PITCH_2"]
for col in remove_list:
    cols.remove(col)

In [6]:
for col in cols:
    q1 = df[col].quantile(.25)
    q3 = df[col].quantile(.75)
    IQR = q3 - q1
    lower = q1 - abs(1.5 * IQR)
    upper = q3 + abs(1.5 * IQR)
    outlier_count = df[(df[col]<lower) | (df[col]>upper)].Dati.count()
    if outlier_count > 0:
        print(col)
        print(outlier_count)
        print("\n")

ENGINE_1_FLOWTEMPA
2


ENGINE_1_FUEL_CONSUMPTION
15173


ENGINE_2_FLOWTEMPA
2


ENGINE_2_FUEL_CONSUMPTION
14298


RATE_OF_TURN
83064


SOG
43729


SOG_SPEEDLOG_TRANS
21373


SPEED_1
1


STW
43057


WIND_SPEED_TRUE
10




### flow temp
Low flow temp happens only when SOG is low, and at a time where outside temperature could be low
Seems reasonable

In [116]:
col = "ENGINE_1_FLOWTEMPA"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
tmp = df[(df[col]<lower) | (df[col]>upper)][["ENGINE_1_FLOWTEMPA", "ENGINE_2_FLOWTEMPA", "Dati", "SOG"]]
tmp[tmp.SOG > 1]

,ENGINE_1_FLOWTEMPA,ENGINE_2_FLOWTEMPA,Dati,SOG
489390,3.6667,3.6667,210214_162700,3.4283
532162,10.8900,10.8450,210325_000100,8.9100


### Fuel Consumption

In [7]:
col = "ENGINE_1_FUEL_CONSUMPTION"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
tmp = df[(df[col]<lower) | (df[col]>upper)]
print(IQR, q1, q3, upper, lower)
tmp[["ENGINE_1_FUEL_CONSUMPTION", "ENGINE_2_FUEL_CONSUMPTION", "POWER_1", "POWER_2", "SOG", "trip_id"]]

178.62380000000007 527.164625 705.7884250000001 973.7241250000002 259.2289249999999


,ENGINE_1_FUEL_CONSUMPTION,ENGINE_2_FUEL_CONSUMPTION,POWER_1,POWER_2,SOG,trip_id
117,214.7655,192.6335,1620.2786,0.0,14.7350,2
129,136.5040,214.6156,964.0048,0.0,9.6150,2
138,199.5683,255.4101,849.7327,0.0,10.0000,2
139,221.0089,266.7087,1089.2406,0.0,7.6033,2
142,150.0446,224.0237,465.7655,0.0,7.7917,2
...,...,...,...,...,...,...
748044,140.5518,174.0472,538.6257,0.0,5.9700,4076
748184,182.1317,241.1633,1065.5867,0.0,11.4167,4078
748185,137.1337,197.7321,655.3271,0.0,7.1917,4078
748704,182.9075,250.7701,707.5318,0.0,8.4533,4080


### POWER_2
observing from visluazation, the range for power 1 and 2 is similar, 
but power 2 has a lot of outliers. \
This is because power 2 has more 0's,
(i.e. engine 2 is used less than engine 1).\
No need to remove these outliers

### RATE_OF_TURN
outliers in rate of turn is also caused by 0 values. \
Inaccurate 0's, drop rate of turn.

In [5]:
df[df.RATE_OF_TURN==0][["Time", "HEADING", "RATE_OF_TURN", "trip_id"]].groupby(["trip_id"]).count()

,Time,HEADING,RATE_OF_TURN
trip_id,,,
0,190119,190119,190119
5,1,1,1
9,1,1,1
10,1,1,1
18,3,3,3
...,...,...,...
4074,109,109,109
4076,106,106,106
4078,101,101,101


In [11]:
df[df.RATE_OF_TURN==0].groupby(["trip_id"]).count()[df[df.RATE_OF_TURN==0].groupby(["trip_id"]).count()>30].dropna()["Dati"]

trip_id
2202    101.0
2203    105.0
2204    106.0
2205    107.0
2206    109.0
        ...  
4074    109.0
4076    106.0
4078    101.0
4080    100.0
4082    102.0
Name: Dati, Length: 1058, dtype: float64

In [45]:
df[df.trip_id==4050][["Time", "HEADING", "RATE_OF_TURN", "trip_id"]]

,Time,HEADING,RATE_OF_TURN,trip_id
322506,1091301.0,179.5,0.0,4050
322507,1091302.0,181.3,0.0,4050
322508,1091303.0,198.5,0.0,4050
322509,1091304.0,226.1,0.0,4050
322510,1091305.0,241.9,0.0,4050
...,...,...,...,...
322603,1091398.0,356.5,0.0,4050
322604,1091399.0,357.2,0.0,4050
322605,1091400.0,3.2,0.0,4050
322606,1091401.0,4.6,0.0,4050


In [57]:
df.drop(["RATE_OF_TURN"], axis=1, inplace=True)

### SOG
outliers are lower SOGs, these low speed values are reasonable to have.

In [118]:
col = "SOG"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
print(lower, upper)
print(df.SOG.min(), df.SOG.max())

16.216749999999998 21.62995
0.0833 21.875


### SOG_SPEEDLOG_TRANS
the extreme values in SOG_SPEEDLOG_TRANS seems reasonable when comparing to corresponding SOG and SOG_SPEEDLOG_LONG

In [126]:
col = "SOG_SPEEDLOG_TRANS"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
print(lower, upper)
print(df.SOG_SPEEDLOG_TRANS.min(), df.SOG_SPEEDLOG_TRANS.max())
tmp["computed_SOG"] = df.SOG_SPEEDLOG_TRANS**2 + df.SOG_SPEEDLOG_LONG**2
tmp["SOG_squred"] = df.SOG ** 2
tmp[(tmp.computed_SOG - tmp.SOG_squred)>3]

-1.11925 1.24755
-4.3033 4.7467


,SOG,SOG_SPEEDLOG_TRANS,SOG_SPEEDLOG_LONG,computed_SOG,SOG_squred


### SPEED_1
the extreme values in speed_1 seems unreasonable when comparing to related fields
remove them  and impute later.

In [129]:
col = "SPEED_1"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
tmp = df[(df[col]<lower) | (df[col]>upper)][["SPEED_1", "SPEED_2", "POWER_1", "POWER_2"]]
tmp

,SPEED_1,SPEED_2,POWER_1,POWER_2
168303,-1750.5303,915.1393,18.9395,1154.2505


In [14]:
df.loc[168303, "SPEED_1"] = np.nan

### STW
Similar to SOG, outliers are lower STWs, these low speed values are reasonable to have.

In [135]:
col = "STW"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
print(lower, upper)
print(df.STW.min(), df.STW.max())

15.958200000000003 22.0518
0.0367 27.72


### WIND_SPEED_TRUE

In [136]:
col = "WIND_SPEED_TRUE"
q1 = df[col].quantile(.25)
q3 = df[col].quantile(.75)
IQR = q3 - q1
lower = q1 - abs(1.5 * IQR)
upper = q3 + abs(1.5 * IQR)
tmp = df[(df[col]<lower) | (df[col]>upper)][["WIND_SPEED", "WIND_SPEED_TRUE", "WIND_ANGLE", "WIND_ANGLE_TRUE"]]
tmp

,WIND_SPEED,WIND_SPEED_TRUE,WIND_ANGLE,WIND_ANGLE_TRUE
45146,57.2433,76.1710,8.0,85.8333
53774,57.3625,76.5931,7.0,76.0359
144429,62.4653,80.6130,8.0,81.0367
333417,59.7640,78.0139,359.0,32.0439
339833,61.3587,77.7006,36.0,-73.9282
404834,65.9267,83.8146,18.0,87.3742
410082,60.5967,78.6006,355.0,64.0122
493413,64.6753,81.5418,21.0,-86.2750
605420,61.2540,77.0383,31.0,-77.2211
727370,59.3053,77.0326,21.0,88.4293


In [15]:
outlier_indexes = list(tmp.index)
for index in outlier_indexes:
    df.loc[index, "WIND_SPEED"] = np.nan
    df.loc[index, "WIND_SPEED_TRUE"] = np.nan

In [58]:
df = df.to_csv("../../df_outlier_removed.csv", index=False)

In [86]:
df = pd.read_csv("../../df_outlier_removed.csv")

In [87]:
# kalman filter to impute missing values
def impute_missing_values(data, transition_matrices, observation_matrices, transition_covariance,
                          observation_covariance, initial_state_mean, initial_state_covariance):
    kf = KalmanFilter(transition_matrices=transition_matrices,
                      observation_matrices=observation_matrices,
                      transition_covariance=transition_covariance,
                      observation_covariance=observation_covariance,
                      initial_state_mean=initial_state_mean,
                      initial_state_covariance=initial_state_covariance)
    
    # Create a mask indicating missing values
    mask = np.isnan(data)
    filtered_state_means, _ = kf.filter(data)
    
    # Replace missing values with imputed values
    imputed_data = data.copy()
    imputed_data[mask] = filtered_state_means[mask]
    
    return imputed_data

### Add an empty line for each missing time within trips

In [88]:
# This 
def missing_time(df):
    missing_dict = {}
    for trip_id in (df.trip_id.unique()):
        tmp_df = df[df.trip_id==trip_id]
        fill_in = tmp_df.Time.max()
        tmp_df["time_up"] = tmp_df["Time"].shift(periods=-1, fill_value=fill_in)
        tmp_df["time_diff"] = tmp_df["time_up"] - tmp_df["Time"]
        missing = tmp_df[tmp_df["time_diff"]>1]
        if len(missing)>0:
            missing_list = []
            for i in list(missing.index):
                start = int(missing.loc[i, "Time"]+1)
                end = int(missing.loc[i, "Time"]+missing.loc[i, "time_diff"])
                missing_list = missing_list + [ x for x in range(start, end)]
            missing_dict[trip_id] = missing_list
            # missing_time = missing_time + list(tmp_df[tmp_df["time_diff"]>1].Time)
    return missing_dict

In [89]:
missing = missing_time(df)

/Users/yim/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/yim/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [90]:
len(missing)

113

In [91]:
for trip_id in missing.keys():
    for time in missing[trip_id]:
        df.loc[len(df)] = {"Time":time, "trip_id":trip_id}
df = df.sort_values("Time").reset_index(drop=True)

### Impute missing

In [92]:
df.columns

Index(['index', 'Dati', 'Time', 'DEPTH', 'ENGINE_1_FLOWTEMPA',
       'ENGINE_1_FUEL_CONSUMPTION', 'ENGINE_2_FLOWTEMPA',
       'ENGINE_2_FUEL_CONSUMPTION', 'HEADING', 'LATITUDE', 'LONGITUDE',
       'PITCH_1', 'PITCH_2', 'POWER_1', 'POWER_2', 'SOG', 'SOG_SPEEDLOG_LONG',
       'SOG_SPEEDLOG_TRANS', 'SPEED_1', 'SPEED_2', 'STW', 'THRUST_1',
       'THRUST_2', 'TORQUE_1', 'TORQUE_2', 'WIND_ANGLE', 'WIND_SPEED',
       'WIND_ANGLE_TRUE', 'WIND_SPEED_TRUE', 'trip_id'],
      dtype='object')

In [93]:
def impute_Dati(df):
    missing = (df[pd.isna(df.Dati)].index)
    for i in missing:
        previous_dt = pd.to_datetime(df.loc[i-1].Dati, format='%y%m%d_%H%M%S')
        dt = (previous_dt +  datetime.timedelta(minutes=1)).strftime('%y%m%d_%H%M%S')
        df.loc[i, "Dati"] = dt
impute_Dati(df)

In [94]:
def impute_mode(df, cols):
    for col in cols:
        mode = df[col].mode()
        df[col] = df[col].fillna(mode)
impute_mode(df, ["DEPTH"])

In [95]:
def impute_mean_within_trips(df, cols):
    for col in cols:
        na_trips = df[df[col].isna()].trip_id.unique()
        for trip in na_trips:
            tmp_df = df[df.trip_id==trip]
            missing = list(tmp_df[tmp_df[col].isna()].index)
            mean = tmp_df[col].mean()
            df.loc[missing, col] = mean
impute_mean_within_trips(df, ["HEADING", "WIND_SPEED", "WIND_SPEED_TRUE", "WIND_ANGLE", "WIND_ANGLE_TRUE"])

In [96]:
df.isna().sum()[df.isna().sum()>0]

DEPTH                        187
ENGINE_1_FLOWTEMPA           187
ENGINE_1_FUEL_CONSUMPTION    187
ENGINE_2_FLOWTEMPA           187
ENGINE_2_FUEL_CONSUMPTION    187
LATITUDE                     187
LONGITUDE                    187
PITCH_1                      187
PITCH_2                      187
POWER_1                      187
POWER_2                      187
SOG                          187
SOG_SPEEDLOG_LONG            187
SOG_SPEEDLOG_TRANS           187
SPEED_1                      188
SPEED_2                      187
STW                          187
THRUST_1                     187
THRUST_2                     187
TORQUE_1                     187
TORQUE_2                     187
dtype: int64

In [97]:
def impute_missing(df, cols):
    for col in cols:
        na_trips = df[df[col].isna()].trip_id.unique()
        for trip in na_trips:
            tmp_df = df[df.trip_id==trip]
            missing = list(tmp_df[tmp_df[col].isna()].index)
            prev_val = df.loc[missing[0]-1, col]
            after_val = df.loc[missing[-1]+1, col]
            if len(missing) > 1:
                impute_diff = (after_val- prev_val)/(missing[-1]-missing[0]+1)
            else:
                impute_diff = (after_val- prev_val)/2
            for i in range(len(missing)):
                df.loc[missing[i], col] = prev_val + impute_diff*(i+1)
impute_missing(df, df.columns)

In [98]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

In [102]:
df.to_csv("../../df_naive_impute.csv", index=False)